# Predicting biomarkers for depletion of glutathione upon use of paracetamol
Thierry D.G.A. Mondeel & Hans V. Westerhoff <br />
University of Amsterdam 2016

Set up Python, load RECON 2, add opthalmic acid reactions (check if needed for RECON 2.2!), add demand reaction for glutathione.

In [1]:
import cobra
from cobra.solvers import get_solver_name
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import parsimonious
from cobrapyTools import *

# Load Recon2
M = cobra.io.load_json_model("../../models/human/RECON_2_2.json")
M.id = 'RECON2'

for r in M.reactions:
    r.id = r.id.replace('lp_','')
    r.id = r.id.replace('_rp_','')
M.repair()

# Adding the opthalmic acid pathway to RECON 2
opa_e = Metabolite('opa_e')
opa_e.name = 'Ophthalmic acid'
opa_c = Metabolite('opa_c')
opa_c.name = 'Ophthalmic acid'

# exchange reaction
EX_opa_e = Reaction('EX_opa_e')
EX_opa_e.add_metabolites({opa_e: -1.0})
EX_opa_e.lower_bound = 0; EX_opa_e.upper_bound = 1000

# transport reaction going outward
OPAtrs = Reaction('OPAtrs')
OPAtrs.add_metabolites({opa_e: 1.0,opa_c: -1.0})
OPAtrs.lower_bound = -1000; OPAtrs.upper_bound = 1000

# glutathione synthase: OPA <-> gluAB (CE1661)
gluABs = Reaction('gluABs')
gluABs.add_metabolites({opa_c: -1.0,M.metabolites.gly_c:-1.0,M.metabolites.CE1661_c: 1.0})
gluABs.lower_bound = -1000; gluABs.upper_bound = 1000

M.add_reactions([EX_opa_e,OPAtrs,gluABs])

# Find all exchange and transport reactions
exchanges = [rxn for rxn in M.reactions if rxn.products == [] ]
exchangesIds = [rxn.id for rxn in exchanges]
transport = [rxn for rxn in M.reactions if len(rxn.get_compartments()) > 1]

# make all exchanges in/out have a bound of 0/1000
for rxn in exchanges:
    if rxn.lower_bound < 0:
        rxn.lower_bound = 0
    
    rxn.upper_bound = 1000     
        
# Add reactions to the model
# demand reaction for gthrd[c] 
DM_gthrd_c = Reaction('DM_gthrd_c')
DM_gthrd_c.add_metabolites({M.metabolites.gthrd_c: -1.0})
# ATP reaction
atp_synthetase_c = Reaction('atp_synthetase_c')
atp_synthetase_c.add_metabolites({M.metabolites.atp_c: -1.0, M.metabolites.adp_c:1.0, 
                                  M.metabolites.pi_c:1.0})
atp_synthetase_c.lower_bound = -1000; atp_synthetase_c.upper_bound = 1000;

M.add_reactions([atp_synthetase_c,DM_gthrd_c])

# set objective
M.reactions.DM_gthrd_c.upper_bound = 1 
M.change_objective('DM_gthrd_c')

In [4]:
model = M.copy()
model = useMedia(model,'human_minimal_ext2',[1,10])
model.change_objective(model.reactions.biomass_reaction)

df = findBiomarkers(model,['EX_5oxpro_e','EX_opa_e','EX_glu_L_e',
                          'EX_cys_L_e','EX_gly_e'],mode='metabolomics',mods=['DM_gthrd_c'],cutoff=0); df

Working on the following reactions:
           Description      Reaction
DM_gthrd_c              gthrd_c --> 




,ID,Name,Prediction,WT,Mutant,Score
0,glu_L_e,L-glutamate(1-),Reduced,"[-1000.0, 51.435]","[-1000.0, 49.478]",0.038
1,gly_e,glycine,Reduced,"[-1000.0, 146.0]","[-1000.0, 143.0]",0.021
2,opa_e,Ophthalmic acid,Elevated,"[-284.406, 2.0]","[-281.667, 2.0]",0.010


In [8]:
model = M.copy()
model = useMedia(model,'Sahoo2012')
model.change_objective(model.reactions.biomass_reaction)

df = findBiomarkers(model,['EX_5oxpro_e','EX_opa_e','EX_glu_L_e',
                          'EX_cys_L_e','EX_gly_e'],mode='metabolomics',mods=['DM_gthrd_c'],eps=12,cutoff=0); df

Working on the following reactions:
           Description      Reaction
DM_gthrd_c              gthrd_c <=> 




,ID,Name,Prediction,WT,Mutant,Score
0,cys_L_e,L-cysteine,Reduced,"[-1000.0, 11.0]","[-1000.0, -1.0]",1.091
1,gly_e,glycine,Reduced,"[-1000.0, 308.0]","[-1000.0, 272.0]",0.117
2,glu_L_e,L-glutamate(1-),Reduced,"[-1000.0, 234.442]","[-1000.0, 209.422]",0.107
3,opa_e,Ophthalmic acid,Elevated,"[-619.436, 5.0]","[-587.592, 5.0]",0.051


In [9]:
model = M.copy()
model = useMedia(model,'human_minimal_ext3')
model.change_objective(model.reactions.biomass_reaction)

df = findBiomarkers(model,['EX_5oxpro_e','EX_opa_e','EX_glu_L_e',
                          'EX_cys_L_e','EX_gly_e'],mode='metabolomics',mods=['DM_gthrd_c'],eps=1,cutoff=0,fracOpt=0.1); df

Working on the following reactions:
           Description      Reaction
DM_gthrd_c              gthrd_c --> 




,ID,Name,Prediction,WT,Mutant,Score
0,glu_L_e,L-glutamate(1-),Reduced,"[-1000.0, 62.535]","[-1000.0, 60.579]",0.031
1,gly_e,glycine,Reduced,"[-1000.0, 184.503]","[-1000.0, 181.503]",0.016
2,opa_e,Ophthalmic acid,Elevated,"[-296.007, 1.921]","[-293.253, 1.921]",0.009
